### 영수증으로 얻을 수 있는 포인트 알아보기

In [21]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

##### 01. 데이터프레임 생성
- 영수증 전처리

In [22]:
receipt_data = '''
6008다 졸고오

세@특보시치시 2

행브로클리,
후레취벤터 [다이
에우 포도1 #
4 세주도콩콜
드준치즈052
'''
receipt = receipt_data.split('\n')
receipt_df = pd.DataFrame(receipt, columns=['title'])
receipt_df['title'] = receipt_df['title'].apply(lambda x: re.sub('[^가-힣 ]', '', x))
receipt_df = receipt_df[receipt_df.title != '']
receipt_df.set_index('title', inplace=True)
receipt_df.reset_index(inplace=True)
receipt_df

,title
0,다 졸고오
1,세특보시치시
2,행브로클리
3,후레취벤터 다이
4,에우 포도
5,세주도콩콜
6,드준치즈


- 리스트 전처리

In [23]:
# 전처리 전 데이터 불러오기
list_df = pd.read_csv('../data/친환경 제품(초안).csv')
# 중복 데이터 제거
list_df.drop_duplicates(subset=['title'], inplace=True)
# 불필요한 문자 제거
list_df['title_mod'] = list_df['title'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))
list_df['title_mod'] = list_df['title_mod'].apply(lambda x: re.sub(r'\[[^)]*\]', '', x))
list_df['title_mod'] = list_df['title_mod'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', '', x))
list_df['point'] = list_df['point'].apply(lambda x: re.sub(r'[^0-9]+', '', x))
list_df['point'] = list_df['point'].astype(int)
# 인덱스 정렬
list_df.set_index('title', inplace=True)
list_df.reset_index(inplace=True)
# 컬럼별 공백 제거
list_df['market'] = list_df['market'].str.lstrip(' ')
list_df['title'] = list_df['title'].str.lstrip(' ')
list_df['title_mod'] = list_df['title_mod'].str.lstrip(' ')
# 최종본으로 저장
# list_df.to_csv('data/eco_product.csv', index=False)
list_df.head()

,title,point,market,title_mod
0,수실류-은행,10,로컬푸드 직매장,수실류은행
1,수실류-호두,10,로컬푸드 직매장,수실류호두
2,수실류-대추,10,로컬푸드 직매장,수실류대추
3,수실류-밤,10,로컬푸드 직매장,수실류밤
4,과실류-기타,10,로컬푸드 직매장,과실류기타


##### 02. 유사도 검사
- TF-IDF 벡터화

In [24]:
tv = TfidfVectorizer()
receipt_tv = tv.fit_transform(receipt_df['title'])
list_tv = tv.transform(list_df['title_mod'])

- 코사인 유사도 계산

In [25]:
cosine_similarities = cosine_similarity(receipt_tv, list_tv)


- 유사도 컬럼으로 추가

In [26]:
receipt_df['cosine_similarity'] = cosine_similarities.max(axis=1)

- 유사한 항목 추가

In [27]:
most_similar_indices = cosine_similarities.argmax(axis=1)
most_similar_items = list_df.loc[most_similar_indices, 'title_mod'].values
receipt_df['most_similar_item'] = most_similar_items
receipt_df['point'] = list_df.loc[most_similar_indices, 'point'].values
receipt_df['market'] = list_df.loc[most_similar_indices, 'market'].values

In [28]:
receipt_df

,title,cosine_similarity,most_similar_item,point,market
0,다 졸고오,0.000000,수실류은행,10,로컬푸드 직매장
1,세특보시치시,0.000000,수실류은행,10,로컬푸드 직매장
2,행브로클리,0.000000,수실류은행,10,로컬푸드 직매장
3,후레취벤터 다이,0.000000,수실류은행,10,로컬푸드 직매장
4,에우 포도,0.707107,유기농 포도,1270,두레생협
5,세주도콩콜,0.000000,수실류은행,10,로컬푸드 직매장
6,드준치즈,0.000000,수실류은행,10,로컬푸드 직매장


In [29]:
# cosine_similarity의 값이 0.5 이상인 경우만 추출
receipt_df = receipt_df[receipt_df['cosine_similarity'] >= 0.5]
receipt_df

,title,cosine_similarity,most_similar_item,point,market
4,에우 포도,0.707107,유기농 포도,1270,두레생협


In [30]:
# 포인트 합계
receipt_df['point'].sum()

1270

def get_similarty(list, receipt)